import required libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


mount google drive because my data is stored there

In [2]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/archive/Training'
test_dir = '/content/drive/MyDrive/archive/Testing'



Mounted at /content/drive


load and preprocess images

In [3]:
image_size = (224, 224)  # Adjust the size according to your requirements
batch_size = 32

# Data augmentation and preprocessing for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for the test set
test_datagen = ImageDataGenerator(rescale=1./255)

# Loading and augmenting the training images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Loading the test images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [6]:
base_model = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
)

model = keras.models.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_3 (Dense)             (None, 4)                 8196      
                                                                 
Total params: 23,595,908
Trainable params: 23,542,788
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

epochs = 10  # Adjust the number of epochs as needed

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)


Epoch 1/10
90/90 [==============================] - 2699s 30s/step - loss: 0.8490 - accuracy: 0.7181 - val_loss: 122.9827 - val_accuracy: 0.2919
Epoch 2/10
90/90 [==============================] - 2506s 28s/step - loss: 0.6122 - accuracy: 0.7889 - val_loss: 1.8160 - val_accuracy: 0.2944
Epoch 3/10
90/90 [==============================] - 2519s 28s/step - loss: 0.3793 - accuracy: 0.8624 - val_loss: 1.4194 - val_accuracy: 0.2690
Epoch 4/10
90/90 [==============================] - 2485s 28s/step - loss: 0.3241 - accuracy: 0.8861 - val_loss: 1.7479 - val_accuracy: 0.2970
Epoch 5/10
90/90 [==============================] - 2540s 28s/step - loss: 0.2603 - accuracy: 0.9059 - val_loss: 5.0843 - val_accuracy: 0.2665
Epoch 6/10
90/90 [==============================] - 2507s 28s/step - loss: 0.2168 - accuracy: 0.9279 - val_loss: 2.2801 - val_accuracy: 0.2843
Epoch 7/10
90/90 [==============================] - 2556s 28s/step - loss: 0.2135 - accuracy: 0.9230 - val_loss: 4.1639 - val_accuracy: 0.25